In [1]:
#                       Borda count
#                       (for Pearson's statistic)
import numpy as np

In [2]:
def sum_permutation_to_borda_count(borda_count, vote_rank, vote_perm):
    """ incrementally update count by borda weighted vote in a subsample of the full borda size

    Args:
        vote_rank: (np.int_(vote_rank)) python rank array same size as borda_count (0 == first, 1 == second,...)
        borda_count: (np.int_(borda_count) ) the current running total of Borda weighted votes
        vote_perm: the sample permutation of the vote ranking

    Returns:
        borda_count: input borda_count with the weighted votes added
    """
    rank_array = np.int_(sorted(np.arange(0, vote_rank.size) + 1, reverse=True))
    borda_count[vote_perm] += rank_array[np.int_(vote_rank)]
    
    return borda_count

In [3]:
#  demonstrate rationale for permuted borda count ranking & correctness of function -- unit test source:
borda_count = np.array([0,0,0,0,0])
vote_rank = np.array([0,3,1,2])
vote_perm = np.array([2,1,4,3])
# with vote ranking: 0 as 4 points, 1 as 3 points,  2 as 2, 3 as 1, (4 is outside the index range)
# the expected outcome is
borda_expected = np.array([0,1,4,2,3]);  # because
# the first member of vote_perm is vote_rank 0 so borda[2] = 4
# the second member of vote_perm is vote_rank 3 so the borda[1] = 1
# the third member of vote_perm is borda_count position four with rank 1 so the borda[4] = 3
# the fourth member of vote_perm is borda_count position three with rank 2 so the borda[3] = 2
# zero (position one) is not in the vote_perm permutation so borda[0] does not change (= 0)
print('borda_count:\t\t{}\nvote_rank:\t\t{}\npermutaiton:\t\t{}\n'.format(borda_count, vote_rank, vote_perm))
print('expected borda:\t\t{}'.format(borda_expected))
borda_returnded = sum_permutation_to_borda_count(borda_count, vote_rank, vote_perm)
print('\nreturned borda:\t\t{}'.format(borda_returnded))

# reiterate voting with a new permutation:
vote_rank = np.array([1,0,2,3])
vote_perm = np.array([1,2,3,0])
print('\n\nreiterate voting with a new permutation:')
print('vote_rank:\t\t{}\npermutaiton:\t\t{}\n'.format(vote_rank, vote_perm))
borda_expected = np.array([1,4,8,4,3]);
borda_returnded = sum_permutation_to_borda_count(borda_count, vote_rank, vote_perm)
print('expected borda:\t\t{}'.format(borda_expected))
print('\nreturned borda:\t\t{}'.format(borda_returnded))

# reiterate voting with a new permutation:
vote_rank = np.array([2,0,1,3]) #2 0 1 3
vote_perm = np.array([0,3,1,2]) #0 3 1 2
print('\n\nreiterate voting with a new permutation:')
print('vote_rank:\t\t{}\npermutaiton:\t\t{}\n'.format(vote_rank, vote_perm))
borda_expected = np.array([3,7,9,8,3]); # 3 7 9 8 3
borda_returnded = sum_permutation_to_borda_count(borda_count, vote_rank, vote_perm)
print('expected borda:\t\t{}'.format(borda_expected))
print('\nreturned borda:\t\t{}'.format(borda_returnded))

borda_count:		[0 0 0 0 0]
vote_rank:		[0 3 1 2]
permutaiton:		[2 1 4 3]

expected borda:		[0 1 4 2 3]

returned borda:		[0 1 4 2 3]


reiterate voting with a new permutation:
vote_rank:		[1 0 2 3]
permutaiton:		[1 2 3 0]

expected borda:		[1 4 8 4 3]

returned borda:		[1 4 8 4 3]


reiterate voting with a new permutation:
vote_rank:		[2 0 1 3]
permutaiton:		[0 3 1 2]

expected borda:		[3 7 9 8 3]

returned borda:		[3 7 9 8 3]


In [8]:
#                      examine assumptions in development versions of function:
c_candidates = 124
n_voters = 1000;
expect_total_count = (sum(np.arange(0, c_candidates) + 1)) * n_voters

borda_count_I = np.int_(np.zeros(c_candidates))
borda_count_S = np.int_(np.zeros(c_candidates))

v1_count = 0
for voter in range(0, n_voters):
    this_vote = np.random.permutation(c_candidates)
    borda_count_I = increment_borda_count(this_vote, borda_count_I)
    borda_count_S = sum_to_borda_count(this_vote, borda_count_S)
#print('borda_count = {}'.format(borda_count))

if expect_total_count != sum(borda_count_I):
    print('\t\t\tBUG borda_count_I')
if expect_total_count != sum(borda_count_S):
    print('\t\t\tBUG borda_count_S')

print('Expected total {} vs I: {} vs S: {}'.format(expect_total_count, sum(borda_count_I), sum(borda_count_S)))

Expected total 7750000 vs I: 7750000 vs S: 7750000


In [5]:
def increment_borda_count(vote, borda_count):
    """ incrementally update the Borda count by one vote
    
    Args:
        vote: ranking array of same size as borda_count 0 is considerd first, 2 second,...
        borda_count: the current running total of Borda weighted votes
    
    Returns:
        borda_count: Borda weighted votes with the input vote added
    """
    rank_array = sorted(np.arange(0, len(vote)) + 1, reverse=True)
    for n in range(0, len(rank_array)):
        borda_count[n] = borda_count[n] + rank_array[vote[n]]
    
    return borda_count

In [6]:
def sum_to_borda_count(vote, borda_count):
    """ incrementally update the Borda count by one vote
    
    Args:
        vote: (np.int_(vote)) python rank array same size as borda_count (0 == first, 1 == second,...)
        borda_count: (np.int_(borda_count) ) the current running total of Borda weighted votes
    
    Returns:
        borda_count: input borda_count with the weighted votes added
    """
    rank_array = np.int_(sorted(np.arange(0, len(vote)) + 1, reverse=True))
    borda_count += rank_array[np.int_(vote)]
    
    return borda_count

In [7]:
#              generate random demonstration data for inspection - mathematical reasoning check:
n_trials = 5
property_size = 5
ord_list = np.arange(0,property_size)
sample_fraction = 0.8
sample_size = round(sample_fraction * property_size)
sample_perm = np.random.permutation(property_size)
sample_perm = sample_perm[0:sample_size]
print('perm: {} w {} elements eg:\n{} in\n{}'.format(sample_perm, len(sample_perm), ord_list[sample_perm], ord_list))

borda_count = np.int_(np.zeros(property_size))
for doda in range(0, n_trials):
    sample_perm = np.random.permutation(property_size)
    sample_perm = sample_perm[0:sample_size]
    vote_rank = np.random.permutation(sample_size)
    print('borda_count:\t\t{}\nvote_rank:\t\t{}\npermutaiton:\t\t{}\n'.format(borda_count, vote_rank, sample_perm))
    borda_count = sum_permutation_to_borda_count(borda_count, vote_rank, sample_perm)
    
print('borda_count sum:\t{}\n'.format(borda_count))

perm: [0 2 3 1] w 4 elements eg:
[0 2 3 1] in
[0 1 2 3 4]
borda_count:		[0 0 0 0 0]
vote_rank:		[3 1 0 2]
permutaiton:		[2 4 0 1]

borda_count:		[4 2 1 0 3]
vote_rank:		[3 2 1 0]
permutaiton:		[2 4 0 1]

borda_count:		[7 6 2 0 5]
vote_rank:		[1 0 2 3]
permutaiton:		[0 4 3 2]

borda_count:		[10  6  3  2  9]
vote_rank:		[0 1 3 2]
permutaiton:		[1 0 4 3]

borda_count:		[13 10  3  4 10]
vote_rank:		[0 1 2 3]
permutaiton:		[2 0 3 1]

borda_count sum:	[16 11  7  6 10]

